In [2]:
from datasets import load_dataset, Dataset, concatenate_datasets
import random
import json
import re
from openai import OpenAI
import bertopic
import datashader
import colorcet
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from umap import UMAP

In [ ]:
class SyntheticDataClient:
    def __init__(self):
        self.prompts_with_context = [
            "Opsummer teksten til en enkelt overskrift",
            "Hvad er tekstens hovedpointe?",
            "Giv mig de 3 vigtigste pointer fra denne tekst",
            "Hvilke kilder eller referencer bruges i denne tekst, og hvordan påvirker de dens troværdighed?",
            "Hvad er den overordnede struktur eller organisering af denne tekst?",
            "Hvad handler denne tekst om? Skriv det med én sætning",
            "Identificer det vigtigste emne i denne tekst."
            "Beskriv indholdet i denne tekst med 1 ord",
            "Lav en gåde der handler om det samme som denne tekst.",
            "Skriv denne tekst om til et digt",
            "Omskriv denne tekst så den har et mere simpelt sprog",
            "Hvilke stilistiske træk kendetegner denne tekst?",
            "Kan du lave denne tekst om, så den er mere formel?",
            "Hvilke alternative titler kunne man give til denne tekst, der stadigvæk fanger dens essens?",
            "Hvordan ville denne tekst blive modtaget af forskellige kulturer eller samfund i forskellige dele af verden?",
            "Hvis denne tekst var en rejse, hvilke destinationer ville den inkludere, og hvad ville være højdepunkterne på rejsen?",
            "Hvad er den overordnede betydning eller relevans af de præsenterede fakta inden for det bredere samfundsmæssige, videnskabelige eller kulturelle kontekst?",
            "Kan du formulere de mest essentielle indsigter fra denne tekst i et mere akademisk sprog?",
            "Jeg forstår ikke denne tekst. Kan du forklare den for mig?",
            "Skriv en nyhedsartikel, der handler om det samme som denne tekst."
        ]

        self.prompt_without_context = [
            "Kan du formulere et spørgsmål og et svar til denne tekst?",
            "Skriv noget, som en bruger kunne spørge en chatbot om og hvad chatbotten ville svare."
        ]

        self.client = OpenAI()
        self.model = "gpt-4"
        self.system_prompt = "You are a helpful assistant."

    def call_llm(self, prompt: str):
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content

    def sample(self, txt: str) -> str:

        if random.choice([True, False]): 
            # Do not include context
            prompt = f"""
            {txt}
            
            {random.choice(self.prompt_without_context)} Skriv det som JSON med formatet: {{"user": "", "answer": ""}}
            """

            response_from_llm = self.call_llm(prompt)

            response = json.loads(response_from_llm)

            return {"prompt": response['user'], "completion": response['answer']}

        else:
            # Include context
            if random.choice([True, False]):
                # Context first
                prompt = f"""
                {txt}

                {random.choice(self.prompts_with_context)}   
                """
            else:
                # Context last
                prompt = f"""
                {random.choice(self.prompts_with_context)} 

                {txt}  
                """

            response = self.call_llm(prompt)

            return {"prompt": prompt, "completion": response}


In [3]:
dawiki = load_dataset('alexandrainst/scandi-wiki', 'da')

In [4]:
embedding_model_name = 'paraphrase-multilingual-mpnet-base-v2'
RANDOM_STATE = 666
np.random.seed(RANDOM_STATE)

sentence_model = SentenceTransformer(embedding_model_name)

docs = dawiki['train']['text']

In [5]:
# embeds = sentence_model.encode(docs, show_progress_bar=True)
# print(embeds.shape)

# np.save(f"dataset/embeddings/{embedding_model_name}", embeds)

embeds = np.load(f"dataset/embeddings/{embedding_model_name}.npy")
print(embeds.shape)

(287216, 768)


In [6]:
with open("dataset/danish_stopwords.txt", "r") as f:
    stopwords = f.readlines()

stopwords = [i.replace('\n', '') for i in stopwords]

vectorizer_model = CountVectorizer(ngram_range=(1,2), stop_words=stopwords, min_df=10)

In [7]:
umap_model = UMAP(
    n_components=2,
    low_memory=True, 
    random_state=RANDOM_STATE
)

topic_model = BERTopic(
    language="multilingual",
    umap_model=umap_model,
    vectorizer_model=vectorizer_model,
    calculate_probabilities=False,
)
topics, _ = topic_model.fit_transform(docs, embeds) # <- takes ~17-18min to run

: 

In [20]:
label_dict = {}
for label in topic_model.generate_topic_labels():
    k,v = label.split('_', 1)
    label_dict[k] = v

topics_with_labels = [label_dict[str(x)] if x != -1 else 'noise' for x in topics]

umap_model = topic_model.umap_model
xy = Dataset.from_dict({
    "x": umap_model.embedding_[:,0], 
    "y": umap_model.embedding_[:,1],
    "topics": topics_with_labels,
})

dawiki_new = concatenate_datasets([dawiki['train'], xy], axis=1)
dawiki_new.save_to_disk('dataset/da-wiki-embed')

In [23]:
import datamapplot
import urllib.parse as urlp


fig, ax = datamapplot.create_interactive_plot(
    np.array([dawiki['x'], dawiki['y']]), 
    dawiki['topics'],
    hover_text=dawiki['title']
    noise_label="noise",
    font_family='Arial',
    on_click="window.open(`http://da.wikipedia.org/wiki/\"{hover_text}\"`)"
    enable_search=True,
    # label_font_size=14,
    # label_margin_factor=3.0
)